# Import

## Choose Model

In [1]:
base_model = "microsoft-deberta-base"
num_epoch = 10
input_strategy = 2
subset_labels = False
# Which exact fold to use
fold = 4

## Labels

In [2]:
labels = ["none", "attribution", "causal", "conditional", "contrast", "description", "equivalence", "fulfillment", "identity", "purpose", "summary", "temporal"]
if subset_labels:
    labels = ["none", "causal", "contrast", "equivalence", "identity", "temporal", "others"]

## Libs

In [3]:
import torch
import numpy as np
import random
import pandas as pd
from IPython.display import display, HTML

## Test Data

In [4]:
data_path = "data/export"
model_path = "data/model"
if subset_labels:
    data_path = "data/export_subset"
    model_path = "data/model_subset"

In [5]:
#test = pd.read_csv("/data/experiments/raring/semantic_storytelling/data/test/data.csv")
test = pd.read_csv(f"{data_path}/{input_strategy}/test.{fold-1}.csv")
origin = test["origin"].tolist()
target = test["target"].tolist()
label = test["label"].tolist()

## Metric

In [6]:
from sklearn.metrics import classification_report
import collections

#classification_threshold = 0.
softmax = torch.nn.Softmax(dim=-1)

def flatten(d, parent_key='', sep='__'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

def compute_metrics(eval_pred):
    global labels
    predictions, true_labels = eval_pred
    # take most probable guess
    predictions = np.argmax(predictions, axis=-1)
    return flatten(classification_report(
        y_true=true_labels,
        y_pred=predictions,
        zero_division=0,
        output_dict=True))

## Model & Tokenizer

In [7]:
model_checkpoint = f"{model_path}/{input_strategy}/{base_model}/epoch_{num_epoch}/fold_{fold-1}/"


from transformers import BertTokenizerFast, DebertaTokenizerFast
# Tokenizer
if "microsoft" in base_model:
    tokenizer = DebertaTokenizerFast.from_pretrained(model_checkpoint)
else:
    tokenizer = BertTokenizerFast.from_pretrained(model_checkpoint)

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
num_labels = len(labels)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
trainer = Trainer(
    model,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


## Create Dataset

In [8]:
class SemanticDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
    
test_encodings = tokenizer(origin, target, truncation=True, padding=True, return_token_type_ids=True)
test_dataset = SemanticDataset(test_encodings, label)

# Predict

In [9]:
prediction_output = trainer.predict(test_dataset)
# display metrics
df = pd.DataFrame([prediction_output.metrics])
display(df.T)

<ipython-input-6-3a5b97d07d22>:11: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.10 it will stop working
  if isinstance(v, collections.MutableMapping):


,0
test_loss,1.239971
test_0__precision,0.883929
test_0__recall,0.873897
test_0__f1-score,0.878884
test_0__support,793.000000
test_1__precision,0.333333
test_1__recall,0.166667
test_1__f1-score,0.222222
test_1__support,6.000000
test_2__precision,0.663636


## Examples

In [10]:
def example_prediction_output(result, origin_list, target_list, label_list, success=True, max_number=100):
    global labels, softmax
    # Real Relation
    data = []
    for n in range(len(origin_list)):
        data.append([origin_list[n], target_list[n], labels[label_list[n]]])
    df_real = pd.DataFrame(data, columns=['Origin', 'Target', 'Label'])
    # Predicted Label
    label_data = []
    for n in range(len(result.label_ids)):
        i = np.argmax(result.predictions[n], axis=-1)
        label_data.append([labels[i]])
    df_pred_labels = pd.DataFrame(label_data, columns=["Prediction"])
    # Prediction Metrics
    pred = softmax(torch.from_numpy(result.predictions))
    df_predictions = pd.DataFrame(pred.numpy(), columns=labels)
    df_predictions = df_predictions.add_prefix("Pred_")
    df = pd.concat([df_real, df_pred_labels, df_predictions], axis=1)
    # Filter complete output to show example
    if success:
        # If possible only show examples with right predicted label that is not none
        df_tmp = df[(df.Label == df.Prediction) & (df.Label != "none")]
        if(df_tmp.size == 0):
            # if there is no match without label "none", print the "none"-matches
            df = df[(df.Label == df.Prediction)]
        else:
            df = df_tmp
    else:
        # If possible only show examples with right predicted label that is not none
        df_tmp = df[(df.Label != df.Prediction) & (df.Label != "none") & (df.Prediction != "none")]
        if(df_tmp.size == 0):
            df_tmp = df[(df.Label != df.Prediction) & ((df.Label != "none") | (df.Prediction != "none"))]
            if(df_tmp.size == 0):
                df = df[df.Label != df.Prediction]
            else:
                df = df_tmp
        else:
            df = df_tmp
    display(HTML(df.head(max_number).to_html()))

### Successes

In [11]:
example_prediction_output(prediction_output, origin, target, label)

,Origin,Target,Label,Prediction,Pred_none,Pred_attribution,Pred_causal,Pred_conditional,Pred_contrast,Pred_description,Pred_equivalence,Pred_fulfillment,Pred_identity,Pred_purpose,Pred_summary,Pred_temporal
52,French police detain 20 terror suspects in dawn raid. Those arrested are suspected of staging armed robberies to finance radical Islamist causes.,"Australian Government passes new terror laws. Obviously, we want to put them through as quickly as possible because we are addressing a terrorist threat.",causal,causal,0.000959,0.000017,0.997116,0.000083,0.000268,0.000250,0.000141,0.000137,0.000041,0.000031,0.000019,0.000938
120,"Brazilian delegation returns from Arafat funeral. The delegation representing Brazil at the funeral of Yasser Arafat returned today, November 13, 2004.","Palestinians to elect new president on January 9. New leadership could prove to be the key to revitalizing the peace process in the Middle East, as both Israel and the United States had refused to work with Arafat.",temporal,temporal,0.000302,0.000042,0.000209,0.000552,0.000096,0.000172,0.000071,0.000545,0.000053,0.000092,0.000020,0.997846
130,"Hamas wins Palestinian election. The Palestinian people faithfully chose the option of resistance and are to fully support it, said Iran's Foreign Ministry Spokesman Hamid-Reza Asefi.","Palestinians to elect new president on January 9. New leadership could prove to be the key to revitalizing the peace process in the Middle East, as both Israel and the United States had refused to work with Arafat.",contrast,contrast,0.092820,0.001127,0.003684,0.000303,0.880639,0.000652,0.013592,0.001228,0.000126,0.003435,0.001048,0.001346
168,Rhode Island District Court freezes Palestinian Authority assets in the US. Yasser Arafat hired lawyer and former attorney general Ramsey Clark as his defense.,"Palestinians to elect new president on January 9. The Haaretz had initially reported that former prime minister Mahmoud Abbas was selected by the Fatah central committee as their candidate for president, but Abbas has denied this, saying, the matter is still being discussed.",temporal,temporal,0.056782,0.000096,0.000311,0.000814,0.000334,0.000593,0.000164,0.000777,0.000089,0.000215,0.000047,0.939776
176,"Hamas wins Palestinian election. However preliminary results based on 95% of the votes cast give Hamas' Change and Reform Party 76 seats, leaving Fatah with 43 seats.","Palestinians to elect new president on January 9. The Haaretz had initially reported that former prime minister Mahmoud Abbas was selected by the Fatah central committee as their candidate for president, but Abbas has denied this, saying, the matter is still being discussed.",causal,causal,0.000341,0.000028,0.998444,0.000107,0.000202,0.000265,0.000169,0.000136,0.000057,0.000046,0.000039,0.000165
179,"Brazilian delegation returns from Arafat funeral. The delegation representing Brazil at the funeral of Yasser Arafat returned today, November 13, 2004.","Palestinians to elect new president on January 9. The Haaretz had initially reported that former prime minister Mahmoud Abbas was selected by the Fatah central committee as their candidate for president, but Abbas has denied this, saying, the matter is still being discussed.",temporal,temporal,0.001251,0.000030,0.000209,0.000262,0.000085,0.000160,0.000050,0.000271,0.000048,0.000062,0.000014,0.997559
182,Palestinians to elect new president on January 9. Acting president Rawhi Fattuh has announced today that Palestinian elections will be held on January 9.,"Hamas wins Palestinian election. Palestine's Central Elections Commission has stated that Israeli authorities prevented electoral arrangements in Jerusalem from meeting the required standards for free, fair and transparent elections.",temporal,temporal,0.001431,0.003342,0.008238,0.312762,0.001691,0.007033,0.001423,0.087183,0.002591,0.004375,0.001622,0.568310
188,"Palestinians to elect new president on January 9. Futtuh, head of the Palestinian parliament, was sworn 

### Fails

In [12]:
example_prediction_output(prediction_output, origin, target, label, False)

,Origin,Target,Label,Prediction,Pred_none,Pred_attribution,Pred_causal,Pred_conditional,Pred_contrast,Pred_description,Pred_equivalence,Pred_fulfillment,Pred_identity,Pred_purpose,Pred_summary,Pred_temporal
20,"European airlines question flight bans. Some of Europe's airlines and airports have called into question whether the mass restrictions imposed on the continent's airspace, due to volcanic ash in the area, are necessary.","Ash-triggered flight disruptions cost airlines $1.7 billion. Restrictions over air travel in Europe have been lifted in many parts of the continent today; three-quarters of the scheduled flights were operating, and most of the European airspace having been opened.",fulfillment,causal,0.000494,0.000384,0.981057,0.001165,0.011745,0.001404,0.000604,0.001557,0.000209,0.000387,0.000348,0.000647
112,"UN criticises Israel's latest settlement plan, angry reaction from US and UK. Neither party should engage in efforts or take actions that could unilaterally pre-empt, or appear to pre-empt, negotiations.","Palestinians to elect new president on January 9. New leadership could prove to be the key to revitalizing the peace process in the Middle East, as both Israel and the United States had refused to work with Arafat.",causal,contrast,0.003899,0.001225,0.000516,0.000332,0.984644,0.000414,0.002076,0.001424,0.000109,0.003369,0.001222,0.000770
123,Israeli PM Olmert to step down in September. Olmert has stated that he would work towards peace as long as I am in my position.,"Palestinians to elect new president on January 9. New leadership could prove to be the key to revitalizing the peace process in the Middle East, as both Israel and the United States had refused to work with Arafat.",equivalence,contrast,0.004423,0.001327,0.000773,0.000412,0.981652,0.000513,0.002061,0.002625,0.000102,0.003758,0.001313,0.001040
127,Top Hamas official seized by Israel. 30 members of Palestinian National Authority parliament have also been detained during the crack-down.,"Palestinians to elect new president on January 9. New leadership could prove to be the key to revitalizing the peace process in the Middle East, as both Israel and the United States had refused to work with Arafat.",causal,contrast,0.003117,0.001144,0.302375,0.001907,0.674689,0.002352,0.001584,0.006217,0.000291,0.001864,0.001155,0.003305
129,"Israeli PM Olmert to step down in September. It's true that Olmert was enthusiastic about the peace process and he spoke about this process with great attention, but it has not achieved any progress or breakthrough,he said.","Palestinians to elect new president on January 9. New leadership could prove to be the key to revitalizing the peace process in the Middle East, as both Israel and the United States had refused to work with Arafat.",equivalence,contrast,0.001910,0.000761,0.000712,0.000241,0.989677,0.000327,0.001660,0.001261,0.000063,0.002097,0.000773,0.000517
131,Palestinians to elect new president on January 9. Acting president Rawhi Fattuh has announced today that Palestinian elections will be held on January 9.,"Palestinian president Abbas has ""no desire"" to seek reelection. Palestinian President Mahmoud Abbas has said that he has no desire of running for re-election in a presidential vote he has called for January.",temporal,causal,0.001370,0.001480,0.875807,0.067055,0.000982,0.009634,0.002087,0.014227,0.002154,0.001564,0.001073,0.022568
177,"Voting underway for successor to Yasser Arafat. He is a moderate and many hope he will be able to establish peace with Israel, who refused to deal with Yassar Arafat because of his support for anti-Israel militants who target civilians.","Palestinians to elect new president on January 9. The Haaretz had initially reported that former prime minister Mahmoud Abbas was selected by the Fatah central committee as their candidate for president, but Abbas has denied this, saying, the matter is still being discussed.",attribution,causal,0.033596,0.005091,0.630340,0.013624,0.016761,0.023

### All Cases

In [13]:
def format_prediction_output(result, origin_list, target_list, label_list):
    global labels
    # Real Relation
    data = []
    for n in range(len(origin_list)):
        data.append([origin_list[n], target_list[n], labels[label_list[n]]])
    df_real = pd.DataFrame(data, columns=['Origin', 'Target', 'Label'])
    # Predicted Label
    label_data = []
    for n in range(len(result.label_ids)):
        i = np.argmax(result.predictions[n])
        label_data.append([labels[i]])
    df_pred_labels = pd.DataFrame(label_data, columns=["Prediction"])
    # Prediction Metrics
    df_predictions = pd.DataFrame(result.predictions, columns=labels)
    df_predictions = df_predictions.add_prefix("Pred_")
    df = pd.concat([df_real, df_pred_labels, df_predictions], axis=1)
    display(HTML(df.to_html()))
    
format_prediction_output(prediction_output, origin, target, label)

,Origin,Target,Label,Prediction,Pred_none,Pred_attribution,Pred_causal,Pred_conditional,Pred_contrast,Pred_description,Pred_equivalence,Pred_fulfillment,Pred_identity,Pred_purpose,Pred_summary,Pred_temporal
0,"31 children rescued as UK police shut down global paedophile ring. Cox himself was found to have 75,960 indecent and explicit images on his computer and police found evidence he supplied at least 11,491 images to other paedophiles over the chatroom, on which he used the identity Son of God. He admitted to nine counts of possessing or distributing images of children.","U.K. National Portrait Gallery threatens U.S. citizen with legal action over Wikimedia images. David Gerard, former Press Officer for Wikimedia UK, the U.K. chapter of the Wikimedia Foundation, which has been involved with the Wikipedia Loves Art contest to create free content photographs of exhibits at the Victoria and Albert Museum, stated on Slashdot that The NPG actually acknowledges in their letter that the poster's actions were entirely legal in America, and that they're making a threat just because they think they can.",none,none,9.086682,-1.982551,-0.026142,-2.080139,-0.027108,-0.864030,-0.608488,-1.740124,-2.483528,-1.175198,-1.624709,-0.879873
1,France knew of and told CIA about al-Qaeda hijack plans prior to 9/11. You have to remember that a plane hijack (in January 2001) did not have the same significance as it did after September 11.,"Bush speaks of goals for U.S. withdrawal from Iraq, decries calls for timetable. A day before the president's address, Sen. Hillary Clinton, in an E-mail to her supporters had said Given years of assurances that the war was nearly over and that the insurgents were in their 'last throes,' this administration was either not being honest with the American people or did not know what was going on in Iraq.",none,none,9.309014,-2.598999,1.008516,-2.218910,-0.210269,-1.033738,-1.029966,-1.777208,-2.887208,-1.876295,-2.377607,0.693762
2,"China sentences six more to death over riots. An employee at the Xinjiang government's news center, who did not give her name, said that the sentencing was not over.","Bush speaks of goals for U.S. withdrawal from Iraq, decries calls for timetable. A day before the president's address, Sen. Hillary Clinton, in an E-mail to her supporters had said Given years of assurances that the war was nearly over and that the insurgents were in their 'last throes,' this administration was either not being honest with the American people or did not know what was going on in Iraq.",none,none,9.129997,-2.031684,-0.002570,-2.062029,0.068284,-0.942828,-0.535198,-1.807686,-2.542833,-1.245571,-1.679294,-0.819736
3,"John Reed on Orwell, God, self-destruction and the future of writing. Just recently, there was some 15-year-old girl who was-- :'JR:' I was just watching something last night, about a pederast in Idaho, who was fine because the statute of limitations was up.","Bush speaks of goals for U.S. withdrawal from Iraq, decries calls for timetable. A day before the president's address, Sen. Hillary Clinton, in an E-mail to her supporters had said Given years of assurances that the war was nearly over and that the insurgents were in their 'last throes,' this administration was either not being honest with the American people or did not know what was going on in Iraq.",none,none,9.256886,-2.021416,-0.198877,-2.048358,-0.047690,-0.837330,-0.607203,-1.727465,-2.512423,-1.184112,-1.745242,-0.621446
4,"Ratko Mladić arrested for war crimes. Serbia's war crimes court ruled that Mladić is fit for trial, despite claims from family and supporters to the contrary.",Belgium indicts former Chad dictator accused of humanitarian crimes. But the following year the Senegalese high court ruled that Habré could not be prosecuted for crimes committed in another country.,none,none,9.229953,-2.316714,0.319665,-2.145272,0.043985,-1.049735,-0.629575,-1.823794,-2.805109,-1.456642,-1.901844,-0.361587
5,Belgium indicts former Chad di

In [14]:
#display(prediction_output)

PredictionOutput(predictions=array([[ 9.0866823e+00, -1.9825505e+00, -2.6142297e-02, ...,
        -1.1751980e+00, -1.6247088e+00, -8.7987286e-01],
       [ 9.3090143e+00, -2.5989990e+00,  1.0085160e+00, ...,
        -1.8762947e+00, -2.3776066e+00,  6.9376194e-01],
       [ 9.1299973e+00, -2.0316844e+00, -2.5697707e-03, ...,
        -1.2455709e+00, -1.6792936e+00, -8.1973648e-01],
       ...,
       [ 3.1565456e+00, -2.7148576e+00, -4.9461734e-01, ...,
        -1.9101609e+00, -3.6177852e+00,  7.8538804e+00],
       [ 9.3813076e+00, -2.2136924e+00, -2.1555258e-01, ...,
        -1.3256136e+00, -1.7885102e+00, -9.8538212e-02],
       [ 8.6687365e+00, -1.8487691e+00, -8.0461375e-02, ...,
        -1.2123958e+00, -1.5557283e+00, -6.6230732e-01]], dtype=float32), label_ids=array([ 0,  0,  0, ..., 10,  0,  0]), metrics={'test_loss': 1.2399710416793823, 'test_0__precision': 0.8839285714285714, 'test_0__recall': 0.8738965952080706, 'test_0__f1-score': 0.8788839568801521, 'test_0__support': 793, '